In [48]:
import os

import pandas as pd

from demo import path

Load `2020-07.csv`

In [ ]:
file_path = os.path.join(path, f"2020-07.csv")

df = pd.read_csv(file_path)

## Assumption
Since the data legend has no clear indication of final destination, it is fair to have assumed the last station of each
day is the final destination, thus in this demo, `HKI` station is filtered out.

In [50]:
final_destination=df.where(df['stationShortCode'] == 'HKI').dropna(subset=['stationShortCode'])
final_destination # dropped all non-final station rows

,Unnamed: 0,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,...,meta_operatorUICCode,meta_operatorShortCode,meta_trainType,meta_trainCategory,meta_commuterLineID,meta_runningCurrently,meta_cancelled,meta_version,meta_timetableType,meta_timetableAcceptanceDate
133,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-01T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.766041e+11,REGULAR,2020-05-15T09:46:07.000Z
267,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-02T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.766041e+11,REGULAR,2020-05-15T09:46:07.000Z
401,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-03T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.766041e+11,REGULAR,2020-05-15T09:46:07.000Z
535,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-04T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.739552e+11,REGULAR,2020-05-15T09:46:07.000Z
669,133.0,HKI,1.0,FI,ARRIVAL,True,True,7.0,False,2020-07-06T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.739738e+11,REGULAR,2020-05-15T09:46:07.000Z
803,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-07T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.739854e+11,REGULAR,2020-05-15T09:46:07.000Z
937,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-08T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.739954e+11,REGULAR,2020-05-15T09:46:07.000Z
1071,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-09T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.766042e+11,REGULAR,2020-05-15T09:46:07.000Z
1205,133.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-10T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.766042e+11,REGULAR,2020-05-15T09:46:07.000Z
1293,87.0,HKI,1.0,FI,ARRIVAL,True,True,8.0,False,2020-07-11T07:52:00.000Z,...,10.0,vr,IC,Long-distance,NaN,False,False,2.740276e+11,REGULAR,2020-05-15T09:46:07.000Z


### Baseline answer, to the minute:
- `differenceinMinutes` column is the timedelta between `actualTime` and `scheduledTime` was calculated by API data provider,
rounded by the minutes.
- For example, 10s is rounded down to 0 minutes, but 36s is rounded up to 1 minute.
- Hence this is at best a quick proxy answer.

In [51]:
mean_delta_in_minutes = final_destination['differenceInMinutes'].mean()
print(mean_delta_in_minutes)

1.7777777777777777


#### Convert `actualTime` and `scheduledTime` into timestamp, thus calculate time_delta in seconds

In [54]:
date_format = "%Y-%m-%dT%H:%M:%S.%fZ"
final_destination['actualTime_timestamp'] = pd.to_datetime(df['actualTime'], format=date_format)
final_destination['scheduledTime_timestamp'] = pd.to_datetime(df['scheduledTime'], format=date_format)

final_destination['differenceInSeconds'] = (final_destination['actualTime_timestamp'] 
                                            - final_destination['scheduledTime_timestamp']).astype('timedelta64[s]')

### A more precise answer, to the second:

In [56]:
mean_delta_in_seconds =(final_destination['actualTime_timestamp']
                        - final_destination['scheduledTime_timestamp']).dt.total_seconds().mean()
print(mean_delta_in_seconds)

101.55555555555556
